In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_validate
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
#from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.linear_model import LassoCV
from sklearn.cluster import KMeans

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import mutual_info_regression

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [3]:
m = train_data.shape[0]

In [4]:
"""
for col in train_data.columns:
    null_percent = train_data[col].isnull().sum() / m
    if null_percent > 0:
        print(col)
        print(null_percent)
"""

In [5]:
# Drop columns with too many missing data and no impact on the sale price
train_data = train_data.drop(columns=["Id"])
test_data = test_data.drop(columns=["Id"])

In [6]:
train_y = train_data['SalePrice']
train_X = train_data.drop(columns=['SalePrice'])
all_data = pd.concat([train_X, test_data],ignore_index=True)

In [7]:
num_col = list(all_data.select_dtypes(include=['int','float']).columns)
obj_col = list(all_data.select_dtypes(include=['object']).columns)

In [8]:
# Specific pre-processing columns

In [9]:
"""
# Columns with meaningful nan: 
# Alley, GarageType, GarageQual, GarageCond, BsmtQual, BsmtCond, FireplaceQu, PoolQC, 
# BsmtFinType1, BsmtFinType2, BsmtExposure, GarageFinish, Fence, MiscFeature
nan_col = ['Alley', 'GarageType', 'GarageQual', 'GarageCond',
           'BsmtQual', 'BsmtCond', 'FireplaceQu', 'PoolQC', 
           'BsmtFinType1', 'BsmtFinType2', 'BsmtExposure',
           'GarageFinish', 'Fence', 'MiscFeature', 'ExterQual', 
           'ExterCond', 'HeatingQC', 'KitchenQual', 'Functional']
all_data[nan_col] = all_data[nan_col].fillna(0)
"""

In [10]:
# For categorical columns with some sort of hierarchy, rank from low to high, and specify if nan means something
# Have to do this manually for both train and test,
# since it's in the data description, and training data might not have all the values

# Alley: nan = no alley access
# GarageType: nan = no garage

# ExterQual: Po -> Fa -> TA -> Gd -> Ex
# ExterCond: Po -> Fa -> TA -> Gd -> Ex
# HeatingQC: Po -> Fa -> TA -> Gd -> Ex
# KitchenQual: Po -> Fa -> TA -> Gd -> Ex

# GarageQual: nan -> Po -> Fa -> TA -> Gd -> Ex
# GarageCond: nan -> Po -> Fa -> TA -> Gd -> Ex
# BsmtQual: nan -> Po -> Fa -> TA -> Gd -> Ex
# BsmtCond: nan -> Po -> Fa -> TA -> Gd -> Ex
# FireplaceQu: nan -> Po -> Fa -> TA -> Gd -> Ex
# PoolQC: nan -> Po -> Fa -> TA -> Gd -> Ex


# BsmtFinType1: nan -> Unf -> LwQ -> Rec -> BLQ -> ALQ -> GLQ
# BsmtFinType2: nan -> Unf -> LwQ -> Rec -> BLQ -> ALQ -> GLQ


# LotShape: IR3 -> IR2 -> IR1 -> Reg
# Utilities: ELO -> NoSeWa -> NoSeWr -> AllPub
# LandSlope: Gtl -> Mod -> Sev
# BldgType: 1Fam -> 2FmCon -> Duplx -> TwnhsE -> Twnhs
# HouseStyle: 1Story -> 1.5Unf -> 1.5Fin -> 2Story -> 2.5Unf -> 2.5Fin -> SFoyer -> SLvl
# BsmtExposure: nan -> No -> Mn -> Av -> Gd
# Electrical: Mix -> FuseP -> FuseF -> FuseA -> SBrkr
# Functional: Sal -> Sev -> Maj2 -> Maj1 -> Mod -> Min2 -> Min1 -> Typ
# GarageFinish: nan -> Unf -> RFn -> Fin
# PavedDrive: N -> P -> Y
# Fence: nan -> MnWw -> GdWo -> MnPrv -> GdPrv


"""
for val in train_data['SaleCondition'].unique():
    print(val)
    mean = train_data.loc[train_data['SaleCondition'] == val, 'SalePrice'].mean()
    print(mean)
"""

In [11]:
"""
o_val1 = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
o_col1 = ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 'FireplaceQu', 'PoolQC']
o_col3 = ['BsmtFinType1', 'BsmtFinType2']
o_val2 = ['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
o_col4 = ['LotShape', 'Utilities', 'LandSlope', 'BldgType',
          'HouseStyle', 'BsmtExposure', 'Electrical', 'Functional', 
          'GarageFinish', 'PavedDrive', 'Fence']
o_val4 = [['IR3', 'IR2', 'IR1', 'Reg'], 
          ['ELO', 'NoSeWa', 'NoSeWr', 'AllPub'],
          ['Gtl', 'Mod', 'Sev'], 
          ['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],
          ['1Story', '1.5Unf', '1.5Fin', '2Story', '2.5Unf', '2.5Fin', 'SFoyer', 'SLvl'],
          [0, 'No', 'Mn', 'Av', 'Gd'],
          ['Mix', 'FuseP', 'FuseF', 'FuseA', 'SBrkr'],
          ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
          [0, 'Unf', 'RFn', 'Fin'],
          ['N', 'P', 'Y'],
          [0, 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']]
"""

In [12]:
"""
all_data[o_col1+o_col3+o_col4] = all_data[o_col1+o_col3+o_col4].fillna(0)
"""

In [13]:
"""
for col in o_col1:
    for i in range(len(o_val1)):
        all_data.loc[all_data[col] == o_val1[i], col] = i+1
"""

In [14]:
"""
for col in o_col3:
    for i in range(len(o_val2)):
        all_data.loc[all_data[col] == o_val2[i], col] = i+1
"""

In [15]:
"""
for i in range(len(o_col4)):
    col = o_col4[i]
    val_col = o_val4[i]
    for j in range(len(val_col)):
        all_data.loc[all_data[col] == val_col[j], col] = j+1
"""

In [16]:
"""
# Check if functions ran corrrectly
for col in o_col1+o_col3+o_col4:
    print(col)
    print(all_data[col].unique())
"""

In [17]:
"""
obj_col = list(all_data.drop(columns=o_col1+o_col3+o_col4).select_dtypes(include=['object']).columns)
"""

In [18]:
"""
for col in obj_col:
    print(col)
    print(all_data[col].unique())
"""

In [19]:
"""
# Categorical columns that are missing some values in both the dataset compared to the data description
# MSZoning, Condition1, Condition2, Exterior1st, Exterior2nd, MiscFeature, SaleType
# Transform to labels based on data description
o_col5 = ['MSZoning', 'Condition1', 'Condition2', 'Exterior1st', 'Exterior2nd', 'MiscFeature', 'SaleType']
o_val5 = [['A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'],
          ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'],
          ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'],
          ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard',
            'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco',
            'VinylSd', 'Wd Sdng', 'WdShing'],
          ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard',
            'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco',
            'VinylSd', 'Wd Sdng', 'WdShing'],
          [0, 'Elev', 'Gar2', 'Othr', 'Shed', 'TenC'],
          ['WD', 'CWD', 'VWD', 'New', 'COD', 'Con', 'ConLw', 'ConLI', 'ConLD', 'Oth']]
"""

In [20]:
"""
for i in range(len(o_col5)):
    col = o_col5[i]
    val_col = o_val5[i]
    for j in range(len(val_col)):
        all_data.loc[all_data[col] == val_col[j], col] = j
    print(all_data[col].unique())
"""

In [21]:
"""
# As shown above, column Exterior2nd contains wrong values
# Changing the wrong values to the closest value on the data description
# Wd Shng -> Wd Sdng -> 15th value in the list in o_val5 for Exterior2nd
# CmentBd -> CemntBd -> 5th value in the list in o_val5 for Exterior2nd
# Brk Cmn -> BrkComm -> 2nd value in the list in o_val5 for Exterior2nd
all_data.loc[all_data['Exterior2nd'] == 'Wd Shng', 'Exterior2nd'] = 15
all_data.loc[all_data['Exterior2nd'] == 'CmentBd', 'Exterior2nd'] = 5
all_data.loc[all_data['Exterior2nd'] == 'Brk Cmn', 'Exterior2nd'] = 2
"""

In [22]:
"""
# Check if above functions are working corectly
for col in o_col5:
    print(col)
    print(all_data[col].unique())
"""

In [23]:
"""
obj_col = list(all_data.drop(columns=o_col1+o_col3+o_col4+o_col5).select_dtypes(include=['object']).columns)
"""

In [24]:
# Transform the rest of the categorical data to numerical labels (ignoring null values for now)
for col in obj_col:
    val = np.stack(train_data[col].unique())
    for i in range(len(val)):
        if (val[i] == 'nan') or (val[i] == 0):
            val = np.delete(val, i)
            break
    for i in range(len(val)):
        all_data.loc[all_data[col] == val[i], col] = i+1 
        #test_data.loc[test_data[col] == val[i], col] = i+1

In [25]:
# Check if all obj_col are labels
obj_col = list(all_data.select_dtypes(include=['object']).columns)
for col in obj_col:
    print(col)
    print(all_data[col].unique())

In [26]:
# Idea:
# Use ML model to find missing values
# Then use xgb to predict

In [27]:
# For each column:
# Get null rows and drop other null columns
# For every other row, drop null columns too
# Use the other rows as train to predict the values of this column

In [28]:
def pre_process(X):
    
    # Fill columns with the least nulls first
    #col_with_null = train_data.isnull().sum()
    #col_with_null = col_with_null[col_with_null > 0]
    #col_with_null = col_with_null.sort_values().index.tolist()
    col_with_null = X.columns[X.isnull().any()].tolist()
    for i in range(len(col_with_null)):
        # col = current target column trying to fill null
        col = col_with_null[i]
        # other_col = other columns with null
        # other_col = np.append(col_with_null[0:i], col_with_null[i+1:len(col_with_null)])
        other_col = col_with_null[i+1:len(col_with_null)]
        
        # test = all rows with col = null
        test = X.loc[X[col].isnull()]
        idx = test.index
        # train = all rows with col != null
        train = X.drop(idx)
        
        # drop other columns with null from test and train
        test = test.drop(columns=other_col)
        train = train.drop(columns=other_col)
        
        # split data into feature and target
        train_y = train[col]
        train_x = train.drop(columns=col)
        test_x = test.drop(columns=col)
        
        
        # outlier detection on training set using Isolation forest
        train_x = pd.concat([train_x, train_y], axis=1)
        iso_forest = IsolationForest()
        pred = iso_forest.fit_predict(train_x)
        drop_idx = np.where(pred == -1)[0]
        train_x = train_x.drop(labels=[x for x in drop_idx if x in train_x.index], axis=0)
        #print(test_x.isnull().any())
        train_y = train_x[col]
        train_x = train_x.drop(columns=col)
        
        """
        m = train_x.shape[0]
        all_x = pd.concat([train_x, test_x],ignore_index=True)
        
        obj_col = list(all_x.select_dtypes(include=['object']).columns)
        
        
        # Pick columns with to many unique values to use as training data for clustering
        clust_cols = []
        for col in obj_col:
            if(len(all_x[col].unique()) > 10):
                clust_cols = clust_cols + [col]
        
        data = all_x[clust_cols]
        kmeans = KMeans(n_clusters=10)
        kmeans.fit(data)
        all_x['Id'] = kmeans.labels_
        obj_col = obj_col + ['Id']
        train_x = all_x.iloc[:m,:]
        test_x  = all_x.iloc[m:,:]
        print(train_x.isnull().any())
        print(train_y.isnull().any())
        print(test_x.isnull().any())
        """
        # Feature Selection
        # Use mutual information to select features
        mi = mutual_info_regression(train_x, train_y)
        drop_idx = np.where(mi == 0)
        train_x = train_x.drop(train_x.columns[drop_idx], axis=1)
        test_x = test_x.drop(test_x.columns[drop_idx], axis=1)
        
        """
        #one-hot-encoding
        obj_col = list(train_x.select_dtypes(include=['object']).columns)
        
        ohe_cols = []
        for col in obj_col:
            if(len(train_x[col].unique()) < 10):
                ohe_cols = ohe_cols + [col]
                
        #Perform onehotencoding
        encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
        encoded_train = encoder.fit_transform(train_x[ohe_cols])
        encoded_test = encoder.transform(test_x[ohe_cols])
        encoded_train = pd.DataFrame(encoded_train)
        encoded_test = pd.DataFrame(encoded_test)
        for col in encoded_train.columns:
            train_x[col] = encoded_train[col]
        for col in encoded_test.columns:
            test_x[col] = encoded_test[col]
        #train_x = pd.concat([train_x, encoded_train], axis=1, ignore_index=True)
        #test_x  = pd.concat([test_x, encoded_test], axis=1, ignore_index=True)
        """
        # Change dataframe to numpy
        y_train = train_y.to_numpy()
        x_train = train_x.to_numpy()
        x_test  = test_x.to_numpy()
        """
        # If the column is categorical, use one-vs-all classifier with stacking classifier
        if(X[col].dtype == 'object'):
            y_train = y_train.astype(float)
            rf = RandomForestClassifier(n_estimators=10)
            svc = SVC()
            estimators = [('rf', rf), ('svc', svc)] 
            model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=2, n_jobs=-1)
            result = cross_validate(model, x_train, y_train, return_estimator=True, cv=2)
            max_index = result['test_score'].argmax()
            model = result['estimator'][max_index]
            if len(np.unique(y_train)) > 1:
                model = OneVsRestClassifier(SVC()).fit(x_train, y_train)
        else:   
        # Otherwise, use stacking regressor
            rf = RandomForestRegressor()
            svr = SVR()
            estimators = [('rf', rf), ('svr', svr)]
            model = StackingRegressor(estimators=estimators, cv=2, n_jobs=-1)
            result = cross_validate(model, x_train, y_train, return_estimator=True, cv=2)
            max_index = result['test_score'].argmax()
            model = result['estimator'][max_index]
        """
        rf = RandomForestRegressor()
        svr = SVR()
        estimators = [('rf', rf), ('svr', svr)]
        model = StackingRegressor(estimators=estimators, cv=2, n_jobs=-1)
        result = cross_validate(model, x_train, y_train, return_estimator=True, cv=2)
        max_index = result['test_score'].argmax()
        model = result['estimator'][max_index]
        test_x[col] = model.predict(x_test)
        X.loc[idx, col] = test_x[col]
        print(X[col].isnull().any())
    return X

In [29]:
all_data = pre_process(all_data)

In [30]:
obj_col = list(all_data.select_dtypes(include=['object']).columns)
obj_col = obj_col + ['MSSubClass', 'YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold']
# MSSubClass, YearBuilt, YearRemodAdd, MoSold, YrSold arre columns that should be categorical
# but have numeric values

In [31]:
for col in obj_col:
    val_list = all_data[col].unique().tolist()
    for val in val_list:
        all_data.loc[all_data[col] == val, col] = round(val)

In [32]:
#train_X = all_data.iloc[:m,:]
#test_data  = all_data.iloc[m:,:]

In [33]:
# According to a post on discussion board, the sale price isn't in normal distribution
# So by applying log transformation, we could make it nearr normal distribution
plt.hist(train_y)

In [34]:
train_y = np.log(train_y)
plt.hist(train_y)

In [35]:
# Pick columns with to many unique values to use as training data for clustering
clust_cols = []
for col in obj_col:
    if(len(all_data[col].unique()) > 10):
        clust_cols = clust_cols + [col]

In [36]:
data = all_data[clust_cols]
kmeans = KMeans(n_clusters=10)
kmeans.fit(data)
all_data['Id'] = kmeans.labels_
#test = test_data[clust_cols]
#test_data['Id'] = kmeans.predict(test)
#all_data = all_data.drop(columns=clust_cols)
#test_data = test_data.drop(columns=clust_cols)

In [37]:
#for col in clust_cols:
#    obj_col.remove(col)
obj_col = obj_col + ['Id']

In [38]:
#Perform onehotencoding
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoded_all = encoder.fit_transform(all_data[obj_col])
#encoded_test = encoder.fit_transform(test_data[obj_col])
encoded_all = pd.DataFrame(encoded_all)
encoded_col = list(encoded_all.columns)
all_data = pd.concat([all_data, encoded_all], axis=1)

In [39]:
#all_data = all_data.drop(columns=obj_col)
#test_data = test_data.drop(columns=obj_col)

In [40]:
"""
# Get skewed data with skewness larger than 0.5, apply log transformation
from scipy.stats import skew
def get_skew(x):
    return skew(x)
"""

In [41]:
"""
skewed = all_data[all_data.columns[~all_data.columns.isin(obj_col + encoded_col)]].apply(get_skew)
skewed = skewed[skewed > 0.5]
for col in skewed.index:
    all_data[col] = np.log1p(all_data[col])
"""

In [42]:
all_data.isnull().any().unique()

In [43]:
"""
# Perform Feature Scaling on non-categorical data
# Standard, MinMax, and MaxAbs suffer from outlies
# Robust and QuantileTransformer don't suffer from outliers, but QuatileTransformer will collapse outliers
# So we choose RobustScaler
from sklearn.preprocessing import RobustScaler
non_cat_col = [i for i in list(all_data.columns) if i not in obj_col]
for col in encoded_col:
    non_cat_col.remove(col)
for col in non_cat_col:
    scaler = RobustScaler()
    all_data[col] = scaler.fit_transform(all_data[[col]])
"""

In [44]:
#all_data.isnull().any().unique()

In [45]:
train_X = all_data.iloc[:m,:]
test_data  = all_data.iloc[m:,:]

In [46]:
X_train = train_X.to_numpy()
#X_train = np.hstack((X_train, encoded_train))
y_train = train_y.to_numpy()
X_test  = test_data.to_numpy()
#X_test = np.hstack((X_test, encoded_test))

In [47]:
# Feature Selection
# Use mutual information to select features
mi = mutual_info_regression(train_X, train_y)
drop_idx = np.where(mi == 0)
all_data = all_data.drop(all_data.columns[drop_idx], axis=1)

In [48]:
"""
# For some reason, BsmtUnfSF contains null, so just drop it for now
all_data.loc[0, all_data.isnull().any()]
all_data = all_data.drop(columns=['BsmtUnfSF'])
"""

In [49]:
"""
# Add polynomial features, only get top 20 in correlation with sale price
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2, interaction_only=True)
poly_feat = pd.DataFrame(poly.fit_transform(all_data.iloc[:,:25]), columns=poly.get_feature_names_out(list(all_data.columns[:25])))
poly_feat_train = poly_feat.iloc[:m, 26:]
corr = poly_feat_train.corrwith(train_y, axis=0)
corr = corr.sort_values(ascending=False)
corr = corr[:20]
poly_feat = poly_feat[corr.index]
all_data = pd.concat([all_data, poly_feat], axis=1)
"""

In [50]:
train_X = all_data.iloc[:m,:]
test_data  = all_data.iloc[m:,:]

In [51]:
# outlier detection on training set using Isolation forest
train_X = pd.concat([train_X, train_y], axis=1)
iso_forest = IsolationForest(random_state=0)
pred = iso_forest.fit_predict(train_X)
drop_idx = np.where(pred == -1)[0]
train_X = train_X.drop(labels=drop_idx, axis=0)

In [52]:
"""
train_y = train_X['SalePrice']
train_X = train_X.drop(columns=['SalePrice'])
all_data = pd.concat([train_X, test_data],ignore_index=True)
for col in all_data.columns:
    if(type(col) == int):
        col_str = str(col)
        all_data = all_data.rename(columns={col: col_str})
"""

In [53]:
"""
train_X = all_data.iloc[:m,:]
test_data  = all_data.iloc[m:,:]
train_X = pd.concat([train_X, train_y], axis=1)
"""

In [54]:
train_y = train_X['SalePrice']
train_X = train_X.drop(columns=['SalePrice'])

In [55]:
X_train = train_X.to_numpy()
y_train = train_y.to_numpy()
X_test = test_data.to_numpy()

In [56]:
# Use grid search to get the optimal hyperparameters
"""
n_estimators = [100,200,300,400,500]
max_depth = [3,4,5,10]
learning_rate=['constant', 'optimal', 'invscaling', 'adaptive']
min_samples_split = [2,4,6,8,10]
min_samples_leaf = [2,4,6,8,10]

coef0=[0.0, 0.1, 0.3, 0.5, 0.7, 1]
shrinking=[True,False]
C = [1, 2, 3]
kernel=['linear', 'poly', 'rbf', 'sigmoid', 'precompute']
degree=[1, 2, 3]
gamma=['scale', 'auto']

selection = ['cyclic', 'random']
eps = [1e-3,  1e-4, 1e-5]
tol=[1e-3, 1e-4, 1e-5, 1e-6]
max_iter = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
cv = [2,3,4,5]

opt_dict = {
            'max_iter': max_iter,
            'tol': tol,
            'eps': eps,
            'tol': tol,
            'cv': cv}

model = LassoCV()

model = RandomizedSearchCV(model, opt_dict, n_jobs=1, cv=2)
model.fit(X_train,y_train)
print(model.best_score_)
print(model.best_params_)
"""

In [57]:
xgbr = xgb.XGBRegressor(subsample=0.9, num_boost_round=25, n_estimators=300, max_depth=3, eta=0.05, colsample_bytree=0.9)
lgbm = LGBMRegressor(n_estimators=300, max_depth=3, learning_rate=0.1)
cb = CatBoostRegressor()
lr = LinearRegression()
kr = KernelRidge()
lcv = LassoCV(tol=1e-05, max_iter=600,eps=1e-05, cv=5)
en = ElasticNet(selection='random', normalize=False, max_iter=500, l1_ratio=0.7, alpha=1)
br = BayesianRidge(tol=0.001, normalize=True, n_iter=300)
svr = SVR(tol=0.001, shrinking=False, max_iter=500, kernel='rbf', gamma='auto', degree=2, coef0=0.3, C=1)
rf = RandomForestRegressor(n_estimators=100, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', max_depth=100)
gb = GradientBoostingRegressor(n_estimators=700, min_samples_split=10, min_samples_leaf=4, max_features='log2', max_depth=30)

In [58]:
 estimators = [
    ('xgbr', xgbr),
    ('lgbm', lgbm),
    ('cb', cb),
    ('rf', rf),
    ('gb', gb)

    #('lr', lr)
    #('kr', kr),
    #('en', en),
    #('br', br)
    #('svr', svr)
    #('lcv', lcv)
]

In [59]:
model = StackingRegressor(estimators=estimators, cv=2, n_jobs=-1)

In [60]:
result = cross_validate(model, X_train, y_train, return_estimator=True)

In [61]:
max_index = result['test_score'].argmax()
model = result['estimator'][max_index]

In [62]:
output = model.predict(X_test)

In [63]:
output = np.exp(output)

In [64]:
index_array = np.zeros(np.shape(output))
for i in range(np.shape(index_array)[0]):
    index_array[i] = 1461+i
index_array = np.reshape(index_array, (np.shape(index_array)[0],1))
output = np.reshape(output, (np.shape(output)[0],1))

In [65]:
index = pd.DataFrame(index_array, columns = ['Id'], dtype=int)
sale = pd.DataFrame(output, columns = ['SalePrice'], dtype=float)
output_df = index.join(sale)

In [66]:
output_df.to_csv('output.csv',index=False)